In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-04-02 12:07:07.594574: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 12:07:07.594610: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 12:07:07.594620: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 12:07:07.594657: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 12:07:07.594670: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


17225924/17225924 [==============================] - 17s 1us/step


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 12:07:28.557991: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6762 - accuracy: 0.7008

2025-04-02 12:07:33.580335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 7s 264ms/step - loss: 0.6762 - accuracy: 0.7008 - val_loss: 0.4998 - val_accuracy: 0.7470
Epoch 2/100
21/21 [==============================] - 5s 235ms/step - loss: 0.4598 - accuracy: 0.8030 - val_loss: 0.3866 - val_accuracy: 0.7952
Epoch 3/100
21/21 [==============================] - 5s 237ms/step - loss: 0.3791 - accuracy: 0.8165 - val_loss: 0.3657 - val_accuracy: 0.7952
Epoch 4/100
21/21 [==============================] - 5s 244ms/step - loss: 0.3030 - accuracy: 0.8692 - val_loss: 0.3875 - val_accuracy: 0.7892
Epoch 5/100
21/21 [==============================] - 5s 241ms/step - loss: 0.2403 - accuracy: 0.9158 - val_loss: 0.3476 - val_accuracy: 0.7952
Epoch 6/100
21/21 [==============================] - 5s 246ms/step - loss: 0.2204 - accuracy: 0.9173 - val_loss: 0.2962 - val_accuracy: 0.8133
Epoch 7/100
21/21 [==============================] - 5s 243ms/step - loss: 0.1872 - accuracy: 0.9323 - val_loss: 0.2744 - val_accuracy: 0.8795
Epoch 8/100

In [12]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 1s 156ms/step - loss: 0.1190 - accuracy: 0.9578
Validation Accuracy: 95.78%


2025-04-02 12:15:48.561524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 1s 160ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.80      0.77       122
           1       0.29      0.23      0.25        44

    accuracy                           0.64       166
   macro avg       0.51      0.51      0.51       166
weighted avg       0.62      0.64      0.63       166


Confusion Matrix:
[[97 25]
 [34 10]]
